# Mise en place

## Importer les librairies necessiares

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import glob
from scipy.fft import rfftfreq, rfft, irfft
from grand import ECEF, Geodetic, GRANDCS, LTP
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit
import os 
import datetime
import grand.dataio.root_trees as rt  # Permet d'extraire les attributs

from grand import ECEF, Geodetic, GRANDCS, LTP
from grand import Geomagnet
from grand import Topography, Reference, geoid_undulation
from grand import topography

Welcome to JupyROOT 6.26/10


## Chargement des données 

In [ ]:
data_dir = "/home/nlebas/grand/examples/grandlib_classes/"

flist=[data_dir+"GP80_20250309_235256_RUN10070_CD_20dB_23DUs_GP43-ChY-X2X-Y2Y-CD-10000-22.root"] # Voici le fichier à charger mis dans une liste


Ouverture du fichier 

In [10]:
# Initialisation des listes pour stocker les données
list_mult = []
list_trigger_time = []
list_trigger_nano = []
list_du_id = []
list_du_long = []
list_du_lat = []
list_du_alt = []
list_traces = []

for fname in flist:
    # Create a DataFile object to load the ROOT data.
    root_file = rt.DataFile(fname)
    
    # Get the total number of entries (each corresponds to a potential cosmic ray trigger event).
    n = root_file.tadc.get_number_of_entries()
    
    # Loop over each event entry.
    for i in range(n):
        # Retrieve ADC and raw voltage data for the current event.
        root_file.tadc.get_entry(i)
        root_file.trawvoltage.get_entry(i)
        
        # Check trigger conditions:
        # - Ensure that not all values in the 10-second trigger pattern are True.
        # - Ensure that at least one value in the channel trigger pattern is True.
        if ~np.all(root_file.tadc.trigger_pattern_10s) and np.any(root_file.tadc.trigger_pattern_ch):
            
            # Skip events with an abnormal GPS trigger time.
            if root_file.tadc.du_seconds[0] > 2045518457:
                continue
            
            # Determine event multiplicity (number of detected elements in this event).
            multiplicity = len(root_file.tadc.du_id)
            list_mult.append(multiplicity)
            
            # Record trigger times (seconds and nanoseconds).
            list_trigger_time.append(root_file.tadc.du_seconds[0])
            list_trigger_nano.append(root_file.tadc.du_nanoseconds[0])
            
            # Record the identifier for the first triggered detection unit.
            list_du_id.append(root_file.tadc.du_id[0])
            
            # Record the GPS data (longitude, latitude, altitude) associated with the event.
            list_du_long.append(root_file.trawvoltage.gps_long[0])
            list_du_lat.append(root_file.trawvoltage.gps_lat[0])
            list_du_alt.append(root_file.trawvoltage.gps_alt[0])
            
            # Extract trace data for the event (raw data and x, y, z channels).
            _traces = []
            _traces.append(root_file.tadc.trace_ch[0][0])
            _traces.append(root_file.tadc.trace_ch[0][1])
            _traces.append(root_file.tadc.trace_ch[0][2])
            _traces.append(root_file.tadc.trace_ch[0][3])
            list_traces.append(_traces)

# Convert lists to NumPy arrays for further analysis.
list_trigger_time = np.array(list_trigger_time)  # Trigger times in seconds.
list_trigger_nano = np.array(list_trigger_nano)  # Trigger times in nanoseconds.
list_du_id = np.array(list_du_id)
list_traces = np.array(list_traces)
list_du_long = np.array(list_du_long)
list_du_lat = np.array(list_du_lat)
list_du_alt = np.array(list_du_alt)
list_mult = np.array(list_mult)
